In [1]:
import pandas as pd
import numpy as np
import os
import env
from acquire import get_zillow

## 1. Acquire data from the cloud database.
>You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

> ***Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.*** 
> * Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) 
> * Only include properties that have a latitude and longitude value.

In [2]:
def get_zillow():
    '''
    Argument: No arguments required
    Actions: 
        1. Checks for the existence of the csv in the current directory
            a. if present:
                i. reads the csv
            b. if not present:
                i. queries MySQL dtabase using the env.py file for the credentials
                ii. saves the csv to the current working directory
    Return: dataframe
    Modules: 
        import env
        import pandas as pd
        import os
    '''
    # a variable to hold the xpected or future file name
    filename = 'zillow.csv'

    # if the file is present in the directory 
    if os.path.isfile(filename):

        # read the csv and assign it to the variable df
        df = pd.read_csv(filename, index_col=0)

        # return the dataframe and exit the funtion
        return df

    # if the file is not in the current working directory,
    else:
        # assign the name of the database to db
        db = 'zillow'

        # use the env.py function to get the url needed from the db
        url = env.get_db_url(db)

        # assign the sql query into the variable query
        query = '''SELECT
  *
FROM properties_2017 p7
  LEFT OUTER JOIN predictions_2017 ON p7.parcelid = predictions_2017.parcelid
  LEFT OUTER JOIN airconditioningtype act ON p7.airconditioningtypeid = act.airconditioningtypeid
  LEFT OUTER JOIN architecturalstyletype ast ON p7.architecturalstyletypeid = ast.architecturalstyletypeid
  LEFT OUTER JOIN buildingclasstype bct ON p7.buildingclasstypeid = bct.buildingclasstypeid
  LEFT OUTER JOIN heatingorsystemtype hst ON p7.heatingorsystemtypeid = hst.heatingorsystemtypeid
  LEFT OUTER JOIN propertylandusetype plut ON p7.propertylandusetypeid = plut.propertylandusetypeid
  LEFT OUTER JOIN storytype st ON p7.storytypeid = st.storytypeid
  LEFT OUTER JOIN typeconstructiontype tct ON p7.typeconstructiontypeid = tct.typeconstructiontypeid
WHERE YEAR(predictions_2017.transactiondate) = 2017 
  AND (latitude IS NOT NULL AND longitude IS NOT NULL);'''

        # query sql using pandas function
        df = pd.read_sql(query, url)
        
        # rid of duplicates
        df = df.loc[:,~df.columns.duplicated()].copy()

        # save the dataframe as a csv to the current working directory
        df.to_csv(filename)

        # returns the dataframe
        return df

In [3]:
# getting df
df = get_zillow()

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1447/91037858.py:24: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename, index_col=0)


In [4]:
df.loc[:,~df.columns.duplicated()].copy()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77574,2864704,10833991,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037132e+13,-0.002245,2017-09-20,Central,NaN,NaN,Central,Condominium,NaN,NaN
77575,673515,11000655,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,6.037101e+13,0.020615,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77576,2968375,17239384,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,6.111008e+13,0.013209,2017-09-21,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
77577,1843709,12773139,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,6.037434e+13,0.037129,2017-09-21,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [5]:
df.columns

Index(['id', 'parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [6]:
# looking at duplicated parcel id's
df.duplicated(subset=['parcelid']).value_counts()

False    77381
True       198
dtype: int64

In [7]:
# replacing duplicates and keeping the first
df = df.sort_values('transactiondate', ascending=False).drop_duplicates(subset=['parcelid'], keep='first')

## 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [8]:
def nulls_by_col(df):
    '''
    This function takes in a dataframe 
    and finds the number of missing values
    it returns a new dataframe with quantity and percent of missing values
    '''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    percent_missing = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': percent_missing})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

In [9]:
df.isnull().sum()

id                              0
parcelid                        0
airconditioningtypeid       52428
architecturalstyletypeid    77175
basementsqft                77331
                            ...  
buildingclassdesc           77366
heatingorsystemdesc         27941
propertylandusedesc             0
storydesc                   77331
typeconstructiondesc        77159
Length: 68, dtype: int64

In [10]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclassdesc,77366,99.980615
buildingclasstypeid,77366,99.980615
finishedsquarefeet13,77340,99.947015
storytypeid,77331,99.935385
basementsqft,77331,99.935385
...,...,...
latitude,0,0.000000
fips,0,0.000000
bedroomcnt,0,0.000000
bathroomcnt,0,0.000000


In [11]:
def nulls_by_row(df):
    '''
    This function takes in a dataframe 
    and finds the number of missing values in a row
    it returns a new dataframe with quantity and percent of missing values
    '''
    num_missing = df.isnull().sum(axis=1)
    percent_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': percent_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [12]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')
    

In [13]:
# summarize(df)

## 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. 
Run the function and document takeaways from this on how you want to handle missing values.

In [14]:
# get the columns
cols = df.columns.to_list()

In [15]:
# check each column for missing values
col_null_count_ls = []
col_null_prop_ls = []
for col in cols:
    col_null_value_count = df[col].isna().sum()
    col_null_value_prop = col_null_value_count/len(df.index)
    col_null_count_ls.append(col_null_value_count)
    col_null_prop_ls.append(col_null_value_prop)
#     print(f'{col} has {col_null_value_count} missing values which is {col_null_value_prop:.2%}')

In [16]:
column_names = ['attribute', 'num_rows_missing', 'pct_rows_missing']

In [17]:
missing_data_cols = pd.DataFrame(np.array([cols, col_null_count_ls, col_null_prop_ls]).T, columns=column_names)

In [18]:
nulls_by_col(df)

,num_rows_missing,percent_rows_missing
buildingclassdesc,77366,99.980615
buildingclasstypeid,77366,99.980615
finishedsquarefeet13,77340,99.947015
storytypeid,77331,99.935385
basementsqft,77331,99.935385
...,...,...
latitude,0,0.000000
fips,0,0.000000
bedroomcnt,0,0.000000
bathroomcnt,0,0.000000


In [19]:
def getDuplicateColumns(df):

	# Create an empty set
	duplicateColumnNames = set()

	# Iterate through all the columns
	# of dataframe
	for x in range(df.shape[1]):

		# Take column at xth index.
		col = df.iloc[:, x]

		# Iterate through all the columns in
		# DataFrame from (x + 1)th index to
		# last index
		for y in range(x + 1, df.shape[1]):

			# Take column at yth index.
			otherCol = df.iloc[:, y]

			# Check if two columns at x & y
			# index are equal or not,
			# if equal then adding
			# to the set
			if col.equals(otherCol):
				duplicateColumnNames.add(df.columns.values[y])

	# Return list of unique column names
	# whose contents are duplicates.
	return list(duplicateColumnNames)


In [20]:
df.columns.to_list()

['id',
 'parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',


In [47]:
cols_to_remove = [x for x in df.columns if x != 'parcelid' and (x[-2:] == 'id') ]

In [48]:
cols_to_remove

['propertycountylandusecode']

In [22]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df

In [23]:
cols_to_remove = [x for x in df.columns if x != 'parcelid' and (x[-2:] == 'id') ]
df = remove_columns(df, cols_to_remove)

In [24]:
df.shape

(77381, 58)

In [25]:
def handle_missing_values(df, prop_required_col = .5, prop_required_row = .75):
    '''
    Actions: this function removes columns and rows based on the proportion of data missing
    
    '''
    # COL
    # setting a threshold for columns based on kwarg
    threshold = int(round(prop_required_col*len(df.index), 0))
    
    # dropping the null values that meet the threshold established
    df.dropna(axis=1, thresh=threshold, inplace=True)
    
    # ROW
    # setting a threshold for the rows based on the kwarg
    threshold = int(round(prop_required_row*len(df.columns), 0))
    
    # dropping the null values that meet the threshold established
    df.dropna(axis=0, thresh=threshold, inplace=True)
    
    return df

In [26]:
# setting proportion
prop_required_col = .5

# creating a threshold amount 
threshold = int(round(prop_required_col*len(df.index), 0))
print(threshold)

38690


In [27]:
len((df.dropna(axis=1, thresh=threshold)).columns)

30

In [28]:
# removing missing values now
df_prep = handle_missing_values(df)

In [29]:
df_prep.shape

(77305, 30)

In [33]:
df['propertylandusedesc']

77578    Single Family Residential
77577    Single Family Residential
77576    Single Family Residential
77575    Single Family Residential
77574                  Condominium
                   ...            
5                      Condominium
4                      Condominium
3        Single Family Residential
2        Single Family Residential
0        Single Family Residential
Name: propertylandusedesc, Length: 77305, dtype: object

In [31]:
summarize(df)

SUMMARY REPORT


Dataframe head: 
       parcelid  bathroomcnt  bedroomcnt  calculatedbathnbr  \
77578  12826780          2.0         3.0                2.0   
77577  12773139          1.0         3.0                1.0   
77576  17239384          2.0         4.0                2.0   

       calculatedfinishedsquarefeet  finishedsquarefeet12    fips  \
77578                        1762.0                1762.0  6037.0   
77577                        1032.0                1032.0  6037.0   
77576                        1612.0                1612.0  6111.0   

       fullbathcnt    latitude    longitude  ...  structuretaxvaluedollarcnt  \
77578          2.0  33937685.0 -117996709.0  ...                    140000.0   
77577          1.0  34040895.0 -118038169.0  ...                     32797.0   
77576          2.0  34300140.0 -118706327.0  ...                     50683.0   

      taxvaluedollarcnt assessmentyear  landtaxvaluedollarcnt  taxamount  \
77578          522000.0         2016.0 

(-48791.03799999999, 4895364.7]    77196
(4895364.7, 9790568.4]                87
(9790568.4, 14685772.1]               14
(14685772.1, 19580975.8]               5
(19580975.8, 24476179.5]               1
(24476179.5, 29371383.2]               0
(29371383.2, 34266586.9]               0
(34266586.9, 39161790.6]               0
(39161790.6, 44056994.3]               0
(44056994.3, 48952198.0]               1
Name: landtaxvaluedollarcnt, dtype: int64 

(-566.6999999999999, 58681.858]    77089
(58681.858, 117343.796]              175
(117343.796, 176005.734]              22
(176005.734, 234667.672]               9
(234667.672, 293329.61]                4
(293329.61, 351991.548]                0
(351991.548, 410653.486]               0
(410653.486, 469315.424]               0
(469315.424, 527977.362]               0
(527977.362, 586639.3]                 1
Name: taxamount, dtype: int64 

(59948352007016.984, 102636920499301.5]    77078
(102636920499301.5, 144902829897603.0]         0
(14490

In [46]:
# getting only single family residences
df = df[df.propertylandusedesc.isin(['Duplex (2 Units, Any Combination)', 'Planned Unit Development', 'Quadruplex (4 Units, Any Combination)', 'Triplex (3 Units, Any Combination', 'Commercial/Office/Residential Mixed Used','Residential General', 'Cooperative', 'Cluster Home'])==False]

In [49]:
import env
import pandas as pd
import os

import scipy.stats as stats
import numpy as np
from sklearn.model_selection import train_test_split

In [50]:
def split_data(df):
    '''
    Arguments: clean dataframe
    Actions: splits Dataframe into a train, validate, and test datasets for explorations
    Returns: train, validate, and test datasets
    Modules:
        1. from sklearn.model_selection import train_test_split
    '''
    # splitting with test focus
    train_val, test = train_test_split(df, train_size=.8, random_state=1017)
    
    #splitting with train/validate focus
    train, validate = train_test_split(train_val, train_size=.7, random_state=1017)

    # exits function and returns train, validate, test
    return train, validate, test

In [ ]:
def clean_zillow():
    '''
    Actions:
    '''
    
    # get data
    df = handle_missing_values(get_zillow())
    
    # dropping duplicates and keeping the first
    df = df.sort_values('transactiondate', ascending=False).drop_duplicates(subset=['parcelid'], keep='first')

    # getting only single family residences
    df = df[df.propertylandusedesc.isin(['Duplex (2 Units, Any Combination)', 'Planned Unit Development', 'Quadruplex (4 Units, Any Combination)', 'Triplex (3 Units, Any Combination', 'Commercial/Office/Residential Mixed Used','Residential General', 'Cooperative', 'Cluster Home'])==False]
    
    # creating list of columns to remove
    cols_to_remove = [x for x in df.columns if x != 'parcelid' and (x[-2:] == 'id') ]
    
    # removing columns
    df = remove_columns(df, cols_to_remove)